<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/%D0%9F%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B_%D1%81_Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai faiss-cpu langchain langchain-community langchain-text-splitters tiktoken python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
import faiss
import pickle
import numpy as np
from docx import Document as DocxDocument

from openai import OpenAI
from google.colab import userdata
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings


In [ ]:
# получаем ключ из секретов
key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = key

# создаем клиента
client = OpenAI()

In [ ]:
doc = DocxDocument("/content/База знаний Postoplan  (часть 1).docx")

text = "\n".join([para.text for para in doc.paragraphs])

print(text[:1000])


#  Пусть соцсети работают на вас

##  Соцсети на автопилоте — создавайте, планируйте и продвигайте свой контент

POSTOPLAN отмечен наградами, ему доверяют 50 000+ специалистов и компаний из 147 стран
76% уменьшили время работы в соцсетях более, чем на 4 часа в неделю
15% увеличили количество клиентов уже в первые месяцы работы с сервисом
55% отметили увеличение охвата постов и рост аудитории

##  Создаём выдающиеся возможности для маркетинга в соцсетях

Гибкие подписки: платите только за подключенные аккаунты, без лишних расходов и переплат
Количество пользователей 
неограниченно: добавляйте коллег, клиентов или сотрудников для командной работы над проектами
Количество проектов 
неограниченно: создавайте сколько угодно рабочих проектов и давайте к ним доступ членам команды
Постинг 24/7: ведите сразу все нужные вам аккаунты на единой платформе
Фоторедактор и библиотека с 2 млн. изображений: сохраняйте свой уникальный фильтр, создавайте визуальный контент в пару кликов
Интеграция со Slac

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_text(text)

print("Количество чанков:", len(chunks))
print(chunks[0])


Количество чанков: 88
#  Пусть соцсети работают на вас

##  Соцсети на автопилоте — создавайте, планируйте и продвигайте свой контент

POSTOPLAN отмечен наградами, ему доверяют 50 000+ специалистов и компаний из 147 стран
76% уменьшили время работы в соцсетях более, чем на 4 часа в неделю
15% увеличили количество клиентов уже в первые месяцы работы с сервисом
55% отметили увеличение охвата постов и рост аудитории

##  Создаём выдающиеся возможности для маркетинга в соцсетях


In [ ]:
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_texts(chunks, embeddings)


/tmp/ipython-input-3184126664.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
vectorstore.save_local("/content/drive/MyDrive/postoplan_faiss")


In [ ]:
loaded_vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/postoplan_faiss",
    embeddings,
    allow_dangerous_deserialization=True
)


In [ ]:
def ask_knowledge_base(question):

    # 1. Находим похожие чанки
    docs = loaded_vectorstore.similarity_search(question, k=3)

    # 2. Формируем контекст
    context = "\n\n".join([doc.page_content for doc in docs])

    # 3. Формируем запрос к модели
    prompt = f"""
Ты консультант по базе знаний.

Используй только информацию из контекста ниже.
Если ответа нет — скажи, что информации недостаточно.

Контекст:
{context}

Вопрос:
{question}

Ответ:
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [ ]:
answer = ask_knowledge_base("Какие преимущества дает Postoplan?")
print(answer)


Postoplan предлагает следующие преимущества:

1. Увеличение аудитории и числа клиентов: 55% пользователей заметили рост аудитории в первые месяцы использования сервиса.
2. Уведомления о состоянии публикаций: Postoplan информирует о загрузке новых постов, что снижает уровень стресса.
3. Экономия времени: отложенный постинг оптимизирует рабочий процесс и освобождает время для других задач.
4. Автоматизация SMM: возможность настройки автопостинга для проведения акций и розыгрышей без риска забыть о важных сообщениях.
5. Банк идей для постов: наличие шаблонов публикаций и инфоповодов, что ускоряет процесс создания контента.


In [ ]:
ask_knowledge_base("Какие функции доступны в бесплатной версии?")


'Информация недостаточно.'